# Complete FAQ Data Ingestion Pipeline

This notebook demonstrates the complete end-to-end process for ingesting FAQ data:

1. **Content Repository**: Ingest source documents (content files)
2. **FAQ Questions & Answers**: Ingest FAQ Q&A pairs
3. **Source Provenance**: Link FAQs to their source documents

**Prerequisites:**
- Database schema must be created
- Sample data files must exist in `data_ingestion/data/`
- Markdown files must exist for checksum computation

**Design Principles:**
- Clear error handling and validation at each step
- Statistics and verification after each ingestion
- Proper dependency order (content → questions → answers → sources)
- Idempotent operations (safe to re-run)

## Setup and Configuration

In [1]:
# Standard library imports
import sys
import hashlib
from pathlib import Path
import sqlite3

# Third-party imports
import pandas as pd
# import numpy as np

# Local imports
from data_ingestion import ContentRepoIngestion, FAQIngestion, FAQSourceIngestion

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

print("✓ Imports successful")

✓ Imports successful


In [2]:
# Configuration
PROJECT_ROOT = Path.cwd().parent if 'granular_impact' in str(Path.cwd()) else Path.cwd()
DB_PATH = PROJECT_ROOT / "databases" / "my_database.db"
DATA_DIR = Path("data_ingestion/data")
MARKDOWN_DIR = DATA_DIR / "markdown"

print(f"Project Root: {PROJECT_ROOT}")
print(f"Database: {DB_PATH}")
print(f"Data Directory: {DATA_DIR}")
print(f"Markdown Directory: {MARKDOWN_DIR}")

# Verify paths exist
assert DB_PATH.exists(), f"Database not found: {DB_PATH}"
assert DATA_DIR.exists(), f"Data directory not found: {DATA_DIR}"
assert MARKDOWN_DIR.exists(), f"Markdown directory not found: {MARKDOWN_DIR}"

print("\n✓ All paths verified")

Project Root: c:\Users\praka\projects\FAQ\FAQ_update
Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\my_database.db
Data Directory: data_ingestion\data
Markdown Directory: data_ingestion\data\markdown

✓ All paths verified


## Helper Functions

In [3]:
def compute_file_checksum(file_path: Path) -> str:
    """
    Compute SHA-256 checksum for a file.
    
    Args:
        file_path: Path to file
        
    Returns:
        64-character hex string (SHA-256)
    """
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")
    
    content = file_path.read_text(encoding='utf-8')
    checksum = hashlib.sha256(content.encode('utf-8')).hexdigest()
    # return checksum
    return "None"


def print_section(title: str):
    """Print a formatted section header."""
    print("\n" + "=" * 80)
    print(f"  {title}")
    print("=" * 80)


def print_result(result: dict, operation: str):
    """Print ingestion result in a formatted way."""
    if result['success']:
        print(f"\n✓ {operation} SUCCESS")
        print(f"  Rows inserted: {result['rows_inserted']}")
        print(f"  Message: {result['message']}")
    else:
        print(f"\n✗ {operation} FAILED")
        print(f"  Message: {result['message']}")
        raise Exception(f"{operation} failed: {result['message']}")


def print_stats(stats: dict, title: str):
    """Print statistics in a formatted table."""
    print(f"\n{title}:")
    for key, value in stats.items():
        print(f"  {key:30s}: {value}")


print("✓ Helper functions defined")

✓ Helper functions defined


---
## Step 1: Ingest Content Repository

The content repository contains source documents (PDFs, markdown files, etc.) that FAQs are derived from.

**Key points:**
- Each row represents a source document
- `content_checksum` is computed from the markdown content
- These checksums will be referenced by FAQ sources later
- Must be ingested first (foundation for everything else)

In [4]:
print_section("STEP 1: CONTENT REPOSITORY INGESTION")

# Load the sample content repo CSV
content_csv_path = DATA_DIR / "sample_content_repo.csv"
content_df = pd.read_csv(content_csv_path)

print(f"\nLoaded {len(content_df)} content records from CSV")
print("\nFirst few records:")
print(content_df[['raw_file_nme', 'title_nme', 'content_checksum']].head())


  STEP 1: CONTENT REPOSITORY INGESTION

Loaded 14 content records from CSV

First few records:
             raw_file_nme                                        title_nme  content_checksum
0   Employee_Handbook.pdf                           Employee Handbook 2024               NaN
1  IT_Security_Policy.pdf       IT Security Policy - Password Requirements               NaN
2  Remote_Work_Policy.pdf  Remote Work Policy - Eligibility and Guidelines               NaN
3      Benefits_Guide.pdf    Employee Benefits Overview - Health Insurance               NaN
4  Payroll_Processing.pdf       Payroll Processing Guide - Tax Withholding               NaN


### Compute Real Checksums

The CSV has placeholder "NA" values. We need to compute real SHA-256 checksums from the markdown files.

In [5]:
# print("\nComputing checksums from markdown files...")

# checksums_computed = 0
# checksum_map = {}  # Store for later use

# for idx, row in content_df.iterrows():
#     markdown_path = Path(row['extracted_markdown_file_path'])
    
#     if markdown_path.exists():
#         checksum = compute_file_checksum(markdown_path)
#         content_df.at[idx, 'content_checksum'] = checksum
#         checksum_map[row['raw_file_nme']] = checksum
#         checksums_computed += 1
#         print(f"  ✓ {row['raw_file_nme']:40s} → {checksum[:16]}...")
#     else:
#         print(f"  ✗ WARNING: Markdown not found: {markdown_path}")
#         # Keep as NA for now

# print(f"\n✓ Computed {checksums_computed} checksums")

# # Show updated data
# print("\nUpdated content with real checksums:")
# print(content_df[['raw_file_nme', 'content_checksum']].head())

### Ingest into Database

In [6]:
# Initialize ingestion
content_ingestion = ContentRepoIngestion(str(DB_PATH))

# Get stats before ingestion
stats_before = content_ingestion.get_stats()
print_stats(stats_before, "Content Repo - BEFORE Ingestion")

# Option: Clear existing content (uncomment if needed)
# print("\nClearing existing content_repo data...")
# clear_result = content_ingestion.clear()
# print(f"  {clear_result['message']}")

# Ingest from DataFrame
print("\nIngesting content_repo data...")
result = content_ingestion.ingest_from_dataframe(content_df)
print_result(result, "Content Repo Ingestion")

# Get stats after ingestion
stats_after = content_ingestion.get_stats()
print_stats(stats_after, "Content Repo - AFTER Ingestion")


Content Repo - BEFORE Ingestion:
  total_records                 : 0
  unique_files                  : 0
  file_types                    : []
  statuses                      : []

Ingesting content_repo data...

✓ Content Repo Ingestion SUCCESS
  Rows inserted: 14
  Message: Successfully inserted 14 rows into content_repo

Content Repo - AFTER Ingestion:
  total_records                 : 14
  unique_files                  : 14
  file_types                    : ['pdf']
  statuses                      : ['Active']


In [7]:
import sqlite3  # Import here to ensure it's available

print(f"\n{'='*80}")
print("  VERIFICATION QUERIES")
print(f"{'='*80}\n")

conn = sqlite3.connect(str(DB_PATH))

# Query 1: Sample checksums
print("Query 1: Sample checksums from database\n")
query = """
SELECT *
FROM content_repo
LIMIT 5
"""
result_df = pd.read_sql_query(query, conn)
print(result_df.to_string(index=False))


  VERIFICATION QUERIES

Query 1: Sample checksums from database

 ud_source_file_id           raw_file_nme raw_file_type  raw_file_version_nbr                                                  raw_file_path                                          extracted_markdown_file_path                                       title_nme content_checksum file_status          created_dt    last_modified_dt
                 1  Employee_Handbook.pdf           pdf                     1  granular_impact/data_ingestion/data/raw/Employee_Handbook.pdf  granular_impact/data_ingestion/data/markdown/Employee_Handbook_v1.md                          Employee Handbook 2024             None      Active 2025-10-27 09:44:40 2025-10-27 09:44:40
                 2 IT_Security_Policy.pdf           pdf                     1 granular_impact/data_ingestion/data/raw/IT_Security_Policy.pdf granular_impact/data_ingestion/data/markdown/IT_Security_Policy_v1.md      IT Security Policy - Password Requirements             None   